In [3]:
from IPython.core.display import HTML
import numpy as np
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from pylab import *
import pysis
import os
import scipy.optimize as sco
import time
import ast
from tkinter import *
from PIL import Image, ImageTk
%matplotlib qt
#argparse (User input)

file = open('cube_data.txt','a')
file.write('test')
file.close()
#pyqt (tkinter alternative)

#continue python implementation, array
#SQL and pgadmin
#correct best fits
#look into parallelization
#think about best implementations for speed/data volume (multi step vs one big database, etc)
#try no fits, time
#argparse and pyqt

#estimate for fitting pixels in every cube


#filter: real amp at channel 69-70 > .01 (too dark)
#pgadmin
#virtual machines


In [4]:
def gaussian(x,mean,sigma,A):
    
    return A*np.exp( -((x-mean)**2.) / (2.*sigma**2.) )

def powerlaw(x,a, k, c):
    
    return a*x**k + c
vims_wave = np.loadtxt('/home/alanyu/Dropbox (MIT)/VIMS_UROP/vims_wave.txt') 

In [5]:
def write_cubes(master_directory,file_name):
    viewing_geometries = os.listdir(master_directory)
    file = open(file_name,'w')
    for folder in viewing_geometries:
        print(f'Working on folder {folder}')
        for cube in os.listdir(folder):
            if cube[0:2] != 'CM' or 'nav' in cube or 'BKGD' in cube:
                continue
            file.write(f'{master_directory}/{folder}/{cube}\n')
    return file_name

In [19]:
def analyze_cube(cube,nav,l,w):
    """
    Given cube data (cube) and navigation information (nav) with dimensions x and y, returns:
    
    1) max/min 2 micron amplitude
    2) max/min sigma
    3) max/min mean 
    4) max/min incidence
    5) max/min emission
    6) max/min phase
    7) max/min lat
    8) max/min lon
    """
        
    #dimensions = (l,w)
    count=0
    band_channels = [29,30,31,32,47,48,49,50,51,52,53,54,55,84,85,86,87,88,89,90] + [i for i in range(160,181)] 
    
    info = np.full((37,l,w),None)
    #0: key (0 = good, 1 = off disk, 2 = low resolution, 3 = bad powerlaw, 4 = dark, 5 = bad incidence/emission), 6 = completely failed 
    #1-6: best fit (1-3: gaussian mean,sigma,amp; 4-6: plaw amp,exp,const)
    #7-21: raw data channel 60:85
    #22-26: phase,emission,incidence, lat, lon
    
    t0=time.time()
    info[7:32]=cube[60:85]
    info[32:35] = nav[1:4]
    info[35:37] = nav[6:8]
    print(f'copy: {time.time()-t0}')
    for i in range(l):
        for j in range(w):
            
            if nav[6,i,j] < -90:  
                count+=1
                info[0,i,j] = 1
                continue
            elif nav[8,i,j]>100000:
                info[0,i,j] = 2
                #continue
            elif cube[69,i,j]<=0.02:
                info[0,i,j] = 4
            elif nav[2,i,j]<70 or nav[3,i,j]>90:
                info[0,i,j] = 5
            
            
            

            try:
                #Fit power law
                p_fit, p_cov = sco.curve_fit(powerlaw, vims_wave[band_channels], cube[band_channels, i,j], p0 = (.005,-2,0), maxfev = 10000)
                p = powerlaw(vims_wave, *p_fit)

                #Subtract power law
                new_data = cube[:,i,j] - p

                #Fit gaussian
                fit,cov = sco.curve_fit(gaussian, vims_wave[64:85],new_data[64:85], \
                                    p0=(2.08,.125,.1), maxfev=100000)

                info[1,i,j] = fit[0]
                info[2,i,j] = fit[1]
                info[3,i,j] = fit[2]
                info[4,i,j] = p_fit[0]
                info[5,i,j] = p_fit[1]
                info[6,i,j] = p_fit[2]
                info[0,i,j] = 0


            except:
                #If powerlaw fit fails, directly fit gaussian
                info[0,i,j] = 3
                try:
                    fit,cov = sco.curve_fit(gaussian, vims_wave[64:85],cube[64:85,i,j], p0=(2.08,.125,.1), maxfev=1000000)
                    info[1,i,j] = fit[0]
                    info[2,i,j] = fit[1]
                    info[3,i,j] = fit[2]
                except:
                    info[0,i,j]=6# print('in')
                    break


    return info

In [20]:
def create_cube_parser(cube_list):
    '''
    Parameters:
    
    master_directory (str): Path to folder with all viewing geometries.
    
    Creates text file with information about each cube (assuming max resolution < 100km): 
    
    1) folder 
    2) name 
    3) max/min 2 micron amplitude
    4) max/min sigma
    5) max/min mean
    6) dimensions
    7) max/min incidence
    8) max/min emission
    9) max/min phase
    10) max/min lat
    11) max/min lon
    
    '''
    count = 0
    #file = open('cube_data.txt','w')
    skipped=set()
    file = open(cube_list,'r')
    
    #print(viewing_geometries)
    while True:
        
        
        cube_path = file.readline().strip()
        
        if not cube_path:
            break
            
        nav = pysis.CubeFile(f'{cube_path[:-4]}_nav.cub').data


        print(f'Working on: {cube_path}')
            
            
        folder = cube_path.split('/')[-2]
        name = cube_path.split('/')[-1]
        
        data = pysis.CubeFile(cube_path).data

        lbl = pysis.CubeFile(cube_path).label
        
        t0=time.time()
        info = analyze_cube(data,nav,lbl['IsisCube']['Instrument']['SwathLength'],lbl['IsisCube']['Instrument']['SwathWidth'])
        t1 = time.time()
        print(t1-t0)
        np.save(os.path.join('cube_info', f'{folder}-{name}.npy'),info)
        print(time.time()-t1)
          
    file.close()  

    return skipped

In [21]:
def create_database(master_directory,file_name='cube_list.txt'):
    create_cube_parser(write_cubes(master_directory,file_name))

In [23]:
#on test folder (one viewing geometry, 11 cubes): 35.3 s ± 1.95 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

create_database('/home/alanyu/Dropbox (MIT)/VIMS_UROP/test')

Working on folder 2005359T100445_2005360T101919
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2005359T100445_2005360T101919/CM_1514277149_1_ir.cub
copy: 0.0013213157653808594
0.008709430694580078
0.005976438522338867
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2005359T100445_2005360T101919/CM_1514280779_1_ir.cub
copy: 0.003013134002685547
0.014861583709716797
0.007208824157714844
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2005359T100445_2005360T101919/CM_1514282576_1_ir.cub
copy: 0.0015194416046142578
2.3320353031158447
0.027874231338500977
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2005359T100445_2005360T101919/CM_1514280052_1_ir.cub
copy: 0.0021393299102783203
2.554499626159668
0.025568485260009766
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2005359T100445_2005360T101919/CM_1514278619_1_ir.cub
copy: 0.0031223297119140625
2.790487289428711
0.02480769157409668
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2005359T100445_2005360T

copy: 0.00128936767578125
0.010245323181152344
0.005487203598022461
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2005359T100445_2005360T101919/CM_1514282576_1_ir.cub
copy: 0.0013692378997802734
1.7638792991638184
0.025494098663330078
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2005359T100445_2005360T101919/CM_1514280052_1_ir.cub
copy: 0.0013561248779296875
1.8045663833618164
0.02517390251159668
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2005359T100445_2005360T101919/CM_1514278619_1_ir.cub
copy: 0.0012788772583007812
1.7766509056091309
0.024132490158081055
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2005359T100445_2005360T101919/CM_1514280412_1_ir.cub
copy: 0.0012392997741699219
1.8212223052978516
0.02828359603881836
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2005359T100445_2005360T101919/CM_1514284191_1_ir.cub
copy: 0.0013585090637207031
3.125314474105835
0.03812241554260254
Working on: /home/alanyu/Dropbox (MIT)/VIMS_UROP/test/2005359

In [ ]:
nav = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/data/2016127T223449_2016130T200342/CM_1841286170_1_ir_nav.cub').data
data = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/data/2016127T223449_2016130T200342/CM_1841286170_1_ir.cub').data
lbl = pysis.CubeFile('/home/alanyu/Dropbox (MIT)/VIMS_UROP/data/2016127T223449_2016130T200342/CM_1841286170_1_ir.cub').label
analyze_cube(data,nav,lbl['IsisCube']['Instrument']['SwathLength'],lbl['IsisCube']['Instrument']['SwathWidth'])

In [ ]:
write_cubes('/home/alanyu/Dropbox (MIT)/VIMS_UROP/data')

In [ ]:
create_cube_parser('/home/alanyu/Dropbox (MIT)/VIMS_UROP/data')

In [ ]:
def read_database(file_name):
    data = {}
    
    file = open(file_name,'r')
    while True:
        try:
            temp = ast.literal_eval(file.readline())
            name = temp['name']
            del(temp['name'])
            data.update({name:temp})
        except Exception as e:
            #print(e)
            break
            
    file.close()
    
    return data


In [ ]:
# d=read_database('test.txt')
# #print(d) 
a='2016127T223449_2016130T200342'
b='2009189T194600_2009189T235739'
c='2005359T100445_2005360T101919'
# vals=[(x,d[x]) for x in d if d[x]['max_amplitude']>1]
# vals
orig = read_database('test.txt')
new = read_database('compare.txt')

[x for x in orig if x not in new and orig[x]['view']==b and orig[x]['dimensions']!=(64,4)]
#len([x for x in new if x not in orig])
#[x for x in vals if x['view'] == a or x['view']==b or x['view']==c]
#[x for x in d if d[x]['view']=='2016127T223449_2016130T200342' and d[x]['max_real_amplitude'] == 0.30726528]
#array 
#each pixel
#better fits
#core, parallel

In [ ]:
def search(database):
    parameters = input('Enter parameters as tuples separated by whitespace. Use None to specify no filter for that parameter. \nOrdering: Real Amplitude, Mean, Sigma, Gaussian Amplitude, Incidence, Emission, Phase, Latitude, Longitude.')
    parameters = [eval(x) for x in parameters]#['None', 'None', 'None','None','None','None','None','None', '(180,360)']]
    #print(parameters)
    #return parameters
    assert len(parameters) == 10, 'Incorrect number of parameters specified'
    code = {0:'real_amplitude',1:'mean',2:'sigma',3:'amplitude',4:'incidence',5:'emission',6:'phase',7:'lat',8:'lon'}
    out = {}
    for cube in database:
        for i in range(len(parameters)):
            if parameters[i] is not None and (database[cube][f'min_{code[i]}']<parameters[i][0] or \
                                              database[cube][f'max_{code[i]}']>parameters[i][1]):
                break
        else:
             out.update({cube:database[cube]})
                
    return out

In [ ]:

%timeit create_cube_parser('/home/alanyu/Dropbox (MIT)/VIMS_UROP/test')
#With fitting
#one directory: 2005359T100445_2005360T101919
#19.2 s ± 1.64 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

#No fitting, but with loops
#16.9 s ± 566 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

#control: just looping through and loading files, no analysis
#15.8 s ± 380 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
d

In [ ]:
class Window(Frame):
    def __init__(self, master = None):
        Frame.__init__(self, master)
        self.master = master
        self.init_window()
    def printtext(e):
        def p():
            print(e.get())
        return p
    def init_window(self):
        self.master.title('GUI')
        
        self.pack(fill=BOTH,expand=1)
        
        #quitButton = Button(self,text='QUIT',command=self.client_exit)
        
        #quitButton.place(x=0,y=0)
        
        menu = Menu(self.master)
        self.master.config(menu=menu)
        
        file = Menu(menu)
        file.add_command(label='Save')
        file.add_command(label='Exit',command=self.client_exit)
        menu.add_cascade(label='File',menu=file)
        
        edit=Menu(menu)
        edit.add_command(label='Show Image',command = self.showImg)
        edit.add_command(label='Show Text',command=self.showText)
        menu.add_cascade(label='Edit',menu=edit)
        
        e=Entry(self)
        e.pack()
        e.focus_set()
        b=Button(self,text='ok')
        b.pack(side='bottom')
        
    
    def showImg(self):
        load=Image.open('2005_3d.png')
        render = ImageTk.PhotoImage(load)
        
        img=Label(self,image=render)
        img.image=render
        img.place(x=0,y=0)
        
    def showText(self):
        text=Label(self,text='Hello World')
        text.pack()
        
    def client_exit(self):
        self.master.destroy()
        
root = Tk()
root.geometry('400x300')
app = Window(root)

root.mainloop()

In [ ]:
import tkinter
class App():
    def __init__(self):
        self.root = tkinter.Tk()
        button = tkinter.Button(self.root, text = 'root quit', command=self.quit)
        button.pack()
        self.root.mainloop()

    def quit(self):
        self.root.destroy()

app = App()

In [ ]:
a = np.array([[[1],[2]],[3]])

In [ ]:
np.save('a.npy',a)
a

In [ ]:
a_load = np.load('a.npy',allow_pickle=True)

In [ ]:
a_load

In [ ]:
x=np.full((3,64,64),4)

In [ ]:
np.max([1,2,3])

In [ ]:
w = np.zeros((10,64,64)
            )

In [ ]:
w[7:9]=x[1:3]

In [ ]:
w

In [ ]:
'hello\n'#.strip()


In [ ]:
s='/home/alanyu/Dropbox (MIT)/VIMS_UROP/2005359T100445_2005360T101919/CM_1514280052_1_ir.cub'
s.split('/')[-1]

In [17]:
test = np.load('/home/alanyu/Dropbox (MIT)/VIMS_UROP/cube_info/2005359T100445_2005360T101919-CM_1514284191_1_ir.cub.npy',allow_pickle=True)
test[5,15]

array([None, None, None, None, None, None, None, -2.2343428732400707,
       -2.2493740515871723, -2.3581545946952596, -2.323856355613393,
       -2.4163049801082246, -2.370074009186231, -2.4712786492576,
       -2.4064590192514674, -2.4119295016264313, -2.5133414820828235,
       -2.4208612354854724, -2.4755192802329904, -2.458534165018508,
       -2.463337187631546, -2.5114895748518076, -2.4213259050183304,
       -2.3432650876288506, -2.4980706020277044, -2.4262568000095595,
       -2.573421819753411, -2.4275245128142755, -2.489052803329075,
       -2.492632686563417, -2.42294643119145, -2.480330723159163,
       -2.5297506277528443, -2.647493470525718, -2.4859598714042463,
       -2.483405534871863, -2.547549697865022, -2.5001005210737053,
       -2.488248929307822, -2.6758751731437433, -2.4880509823467407,
       -2.39666775871174, -2.408673772286023, -2.4236426509579663,
       -2.3930635753805167, -2.3837957701951265, -2.359772689689834,
       -2.2823970043510178, -2.2794345372

In [ ]:
for i in range(10):
    if i<5:
        continue
    if i==6:
        pass
    if i==7:
        pass
    else:
        print(i,'hi')